# Inspecting Fields

Sometimes, we may want to inspect our model Fields, or an instance's fields.

We've already seen one way to do it for model fields:

In [1]:
from pydantic import BaseModel

In [2]:
class Circle(BaseModel):
    center_x: int = 0
    center_y: int = 0
    radius: int = 1
    name: str | None = None

In [3]:
Circle.model_fields

{'center_x': FieldInfo(annotation=int, required=False, default=0),
 'center_y': FieldInfo(annotation=int, required=False, default=0),
 'radius': FieldInfo(annotation=int, required=False, default=1),
 'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}

As we progress in this course, we'll start seeing a lot more data appearing in the `FieldInfo` objects.

We can also examine the fields in a model instance.

Of course we can do this by looking at the representation, or serializing the data:

In [4]:
c1 = Circle(radius=2)
c1

Circle(center_x=0, center_y=0, radius=2, name=None)

In [5]:
c2 = Circle(name="Unit circle")
c2

Circle(center_x=0, center_y=0, radius=1, name='Unit circle')

In [6]:
c1.model_dump()

{'center_x': 0, 'center_y': 0, 'radius': 2, 'name': None}

In [7]:
c2.model_dump()

{'center_x': 0, 'center_y': 0, 'radius': 1, 'name': 'Unit circle'}

As you can see, we get a view of all the fields in the model - some of which were populated by data, and some from defaults.

We might however be interested in knowing which fields were populated from data, and which ones from the defaults.

In [8]:
c1.model_fields_set

{'radius'}

In [9]:
c2.model_fields_set

{'name'}

As you can see, we get a set containing the field names that were populated from data only.

If we want to know which fields were populated from defaults, there is no direct function to do so, but it is enough to recover.

Remember that `model_fields` is a dictionary with all the field names as keys.

So, to get a list of the fields that were populated from defaults, we start with all the field names and substract from it the fields that were populated from data:

In [10]:
c1.model_fields.keys() - c1.model_fields_set

/tmp/ipython-input-2290803691.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  c1.model_fields.keys() - c1.model_fields_set


{'center_x', 'center_y', 'name'}

Knowing what fields were defined from the deserialized data can be helpful in certain cases.

Suppose we have an API endpoint that users can post to, and send us a JSON payload. Some of the items in the payload are optional, and on our end we'll populate the defaults during validation.

However, we may want to echo the user's payload back to them - if we just echoed back the serialized model as-is, it would include the defaults that were injected into the model instance.

That might be perfectly fine, but we may have a requirement to return the model omitting the default values that were injected. In this case we can make use of the `model_fields_set`.

Let's see an example of this.

You should remember that when serializing a model, we can choose which fields to include or exclude.

In [11]:
class Model(BaseModel):
    field_1: int = 1
    field_2: int | None = None
    field_3: str
    field_4: str | None = "Python"

In [12]:
m1 = Model(field_3="m1")
m2 = Model(field_1=1, field_2=None, field_3="m2", field_4="Python")
m3 = Model(field_1=10, field_2=20, field_3="m3", field_4="Pydantic")

In [13]:
m1.model_dump()

{'field_1': 1, 'field_2': None, 'field_3': 'm1', 'field_4': 'Python'}

And if we only want to see the fields that were actually set via the deserialized data:

In [14]:
m1.model_dump(include=m1.model_fields_set)

{'field_3': 'm1'}

Similarly with the other model instances:

In [15]:
m2.model_dump(include=m2.model_fields_set)

{'field_1': 1, 'field_2': None, 'field_3': 'm2', 'field_4': 'Python'}

In [16]:
m3.model_dump(include=m3.model_fields_set)

{'field_1': 10, 'field_2': 20, 'field_3': 'm3', 'field_4': 'Pydantic'}

In [17]:
m3.model_dump(exclude_unset=True)

{'field_1': 10, 'field_2': 20, 'field_3': 'm3', 'field_4': 'Pydantic'}

In [18]:
m1.model_dump(exclude_unset=True)

{'field_3': 'm1'}